In [ ]:
from maths import abs

def find_seed(g, c=0, eps=2**(-26)):
    
    """On réalise une dichotomie pour trouver une racine de g(t)-c"""
    
    a = 0
    b = 1
    t = (a+b)/2
    
    if g(0) > g(1):
        if c > g(0) or c < g(1):
            return(None)
        
        while abs(g(t)-c) > eps:
            if g(t) > c:
                a = t
            else:
                b = t
            t = (a+b)/2

    else :
        if c < g(0) or c > g(1):
            return(None)
        
        while abs(g(t)-c) > eps:
            if g(t) > c:
                b = t
            else:
                a = t
            t = (a+b)/2
            
    return(t)


L'algorithme précédent fonctionne, mais il ne permet pas de traiter tous les types de courbes (ligne de niveau qui fait "demi-tour"). Il faut donc modifier l'algorithme en passant par une autre méthode. Nous allons utiliser le gradient de la fonction f : les lignes de niveau sont perpendiculaires au gradient. A chaque point, on calcule le gradient local [a;b] et on veut que le nouveau point [x[n];y[n]] vérifie deux conditions :
a*x[n] + b*y[n] = 0 (ligne de niveau perpendiculaire au gradient)
(x[n]-x[n-1])^2 + (y[n]-y[n-1])^2 = delta^2 (le nouveau point appartient au cercle de centre l'ancien point ; ainsi on est sûrs que la distance entre deux points sera de delta). On va résoudre ce système à l'aide des polynômes Python.
Il va falloir choisir entre les deux solutions proposées : pour ce faire on regarde quelle solution  


In [ ]:
def le_plus_loin(X, Y, x1, x2, y1, y2):
    """On va faire appel à cette fonction pour déterminer laquelle des deux solutions garder"""
    
    x = X[-2]
    y = Y[-2]
    d1 = (x1-x)**2 + (y1-y)**2
    d2 = (x2-x)**2 + (y2-y)**2
    if d1 > d2:
        return(x1, y1)
    return(x2, y2)

In [ ]:
def grad_f(f,x,y):
    """La fonction pour calculer le gradient d'une fonction à deux variables"""
    
            g = autograd.grad
            return(np.r_[g(f,0)(x,y),g(f,1)(x,y)])

In [ ]:
def simple_contour(f, c=0.0, delta=0.01):
    X = []
    Y = []
    g = lambda t: f(0,t)
    X.append(0)
    Y.append(find_seed(g, c, 2**(-26))
    if Y[-1] == None:
        return([], [])
    
    for k in range(100): #condition d'arrêt, pas encore optimale
        
        a,b = grad_f(X[-1],Y[-1])
        x0 = X[-1]
        y0 = Y[-1]
        
        if a != 0 :
             p = Polynomial(-delta**2+y0**2+x0**2, 2*b*x0/a-2*y0, b**2/a**2+1)
             y1, y2 = p.roots()
             
             if y1.imag != 0 or y2.imag != 0:
                 return(X,Y) #il n'y a plus de solution réelle, donc le système n'a plus de solutions
             
             x1 = -b*y1/a
             x2 = -b*y2/a
             x, y = le_plus_loin(X, Y, x1, x2, y1, y2)
             X.append(x)
             Y.append(y)

        elif b != 0:
             Y.append(0)
             p = Polynomial(y0**2-delta**2, -2*x0, 1)
             x1, x2 = p.roots()
             
             if x1.imag != 0 or x2.imag != 0:
                 return(X,Y)
             
             x = le_plus_loin(X, Y, x1, x2, 0, 0)
             X.append(x)
             
        elif b == 0:
             return(X, Y)
    
    return(X, Y)
             
               